In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/cornell_movie_dialogs_corpus.zip" -d "/content/cornell_movie"

Archive:  /content/drive/MyDrive/cornell_movie_dialogs_corpus.zip
   creating: /content/cornell_movie/cornell movie-dialogs corpus/
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/.DS_Store  
   creating: /content/cornell_movie/__MACOSX/
   creating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/
  inflating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/chameleons.pdf  
  inflating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_lines.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: /content/cornell_movie/cornell

In [ ]:
import numpy as np
import pandas as pd
import string
import pickle
import operator
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import codecs

with codecs.open("/content/cornell_movie/cornell movie-dialogs corpus/movie_lines.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    conversations = []
    for line in lines:
        data = line.split(" +++$+++ ")
        conversations.append(data)

In [ ]:
conversations[:6]

[['L1045', 'u0', 'm0', 'BIANCA', 'They do not!'],
 ['L1044', 'u2', 'm0', 'CAMERON', 'They do to!'],
 ['L985', 'u0', 'm0', 'BIANCA', 'I hope so.'],
 ['L984', 'u2', 'm0', 'CAMERON', 'She okay?'],
 ['L925', 'u0', 'm0', 'BIANCA', "Let's go."],
 ['L924', 'u2', 'm0', 'CAMERON', 'Wow']]

In [ ]:
chats = {}
for tokens in conversations:
    if len(tokens) > 4:
        idx = tokens[0][1:]
        chat = tokens[4]
        chats[int(idx)] = chat

In [ ]:
sorted_chats = sorted(chats.items(), key = lambda x: x[0])
#sorted_chats

In [ ]:
conves_dict = {}
counter = 1
conves_ids = []
for i in range(1, len(sorted_chats)+1):
    if i < len(sorted_chats):
        if (sorted_chats[i][0] - sorted_chats[i-1][0]) == 1:
            if sorted_chats[i-1][1] not in conves_ids:
                conves_ids.append(sorted_chats[i-1][1])
            conves_ids.append(sorted_chats[i][1])
        elif (sorted_chats[i][0] - sorted_chats[i-1][0]) > 1:            
            conves_dict[counter] = conves_ids
            conves_ids = []
        counter += 1
    else:
        pass

In [ ]:
context_and_target = []
#dicarding dialogue without a reply
for conves in conves_dict.values():
    if len(conves) % 2 != 0:
        conves = conves[:-1]
    for i in range(0, len(conves), 2):
        context_and_target.append((conves[i], conves[i+1]))

In [ ]:
context_and_target=context_and_target[:10000]

In [ ]:
context, target = zip(*context_and_target)

In [ ]:
context = list(context)
target = list(target)

In [ ]:
context[:5]

['Did you change your hair?',
 'I missed you.',
 'It was a bratwurst.  I was eating lunch.',
 'You the new guy?',
 "C'mon.  I'm supposed to give you the tour."]

In [ ]:
target[:5]

['No.',
 'It says here you exposed yourself to a group of freshmen girls.',
 'With the teeth of your zipper?',
 'So they tell me...',
 'So -- which Dakota you from?']

In [ ]:
import re
MAX_LEN = 12

In [ ]:
def clean_text(text):
    # Clean text by removing unnecessary characters and altering the format of words

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [ ]:
tidy_context = []
for conve in context:
    text = clean_text(conve)
    tidy_context.append(text)
tidy_target = []
for conve in target:
    text = clean_text(conve)
    tidy_target.append(text)

In [ ]:
bos = "<START> "
eos = " <STOP>"
final_target = [bos + conve + eos for conve in tidy_target] 
encoder_inputs = tidy_context
decoder_inputs = final_target

In [ ]:
encoder_inputs[0]

'did you change your hair'

In [ ]:
with open("encoder_inputs.txt", "w") as output:
  for i in encoder_inputs:
    output.write('%s\n' % i)

In [ ]:
with codecs.open("encoder_inputs.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    encoder_text = []
    for line in lines:
        data = line.split("\n")[0]
        encoder_text.append(data)

In [ ]:
len(encoder_text)

10001

In [ ]:
encoder_text[:5]

['did you change your hair',
 'i missed you',
 'it was a bratwurst  i was eating lunch',
 'you the new guy',
 "c'mon  i am supposed to give you the tour"]

In [ ]:
with open("decoder_inputs.txt", "w") as output:
  for i in decoder_inputs:
    output.write('%s\n' % i)

In [ ]:

with codecs.open("decoder_inputs.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    decoder_text = []
    for line in lines:
        data = line.split("\n")[0]
        decoder_text.append(data)

In [ ]:
len(decoder_text)

10001

**Making Vocabulary**

In [ ]:
full_text = encoder_text + decoder_text

In [ ]:
dictionary = []
for text in full_text:
    words = text.split()
    for i in range(0, len(words)):
        if words[i] not in dictionary:
            dictionary.append(words[i])
# save this dict takes time to run

In [ ]:
full_text[0]

'did you change your hair'

In [ ]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 2000
tokenizer = Tokenizer(num_words=VOCAB_SIZE)

In [ ]:
tokenizer.fit_on_texts(full_text)
word_index = tokenizer.word_index
len(word_index)

13434

In [ ]:
index2word = {}
for k, v in word_index.items():
    if v < 2001:
        index2word[v+3] = k
    if v > 2001:
        continue

In [ ]:
index2word[1] = bos
index2word[3] = eos
index2word[0]= 'padding'
index2word[2]= 'unk'

In [ ]:
word2index = {}
for k, v in index2word.items():
    word2index[v] = k

In [ ]:
len(word2index) == len(index2word)

True

In [ ]:
encoder_sequences = tokenizer.texts_to_sequences(encoder_text)

In [ ]:
decoder_sequences = tokenizer.texts_to_sequences(decoder_text)

In [ ]:
for seqs in encoder_sequences:
    for seq in seqs:
        if seq > 2000:
            print(seq)
            break

In [ ]:
VOCAB_SIZE = len(index2word) + 1
VOCAB_SIZE

2005

In [ ]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 20
encoder_input_data = pad_sequences(encoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
decoder_input_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')

In [ ]:
decoder_input_data

array([[  2,  30,   1, ...,   0,   0,   0],
       [  2,   9, 262, ...,   0,   0,   0],
       [  2,  34,   5, ...,   0,   0,   0],
       ...,
       [  2,  11,   7, ...,   0,   0,   0],
       [  2,  71,   3, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int32)

In [ ]:

num_samples = len(encoder_sequences)
decoder_output_data = np.zeros((num_samples, MAX_LEN, VOCAB_SIZE), dtype="float32")

In [ ]:
# one hot encoding the tokens:
for i, seqs in enumerate(decoder_input_data):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data[i][j][seq] = 1

In [ ]:
# while fine tuning, i'll have to add the new words which are already not in word2idx to it, 

In [ ]:
!unzip "/content/drive/MyDrive/glove.6B.50d.txt.zip" -d "/content/glove"

Archive:  /content/drive/MyDrive/glove.6B.50d.txt.zip
  inflating: /content/glove/glove.6B.50d.txt  


In [ ]:
embeddings_index = {}
with open('/content/glove/glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print("Glove Loaded!")

Glove Loaded!


In [ ]:
embedding_dimention = 50
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
embedding_matrix = embedding_matrix_creater(50, word_index=word2index)

## Baseline Model

In [ ]:
# from keras.layers import Input, Dense, LSTM, TimeDistributed
# from keras.models import Model
import numpy as np
import pandas as pd
import pickle
import spacy
nlp = spacy.load('en')
import itertools
import collections

# Keras and Tensorflow
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import model_from_json
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers.core import Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers import LSTM, TimeDistributed, RepeatVector, Input

import tensorflow as tf

# Reference local code
import os
import sys
from os.path import join
from pathlib import Path
sys.path.append(join(os.getcwd(), 'src'))

%load_ext autoreload
%autoreload 2
#from utils import preprocessing
#from model.textGenModel import TextGenModel

In [ ]:
hidden_size = 512
batch_size = 16
stateful = False

In [ ]:
# model.layers[0].trainable = True

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
encoder_input_data

array([[ 42,   3, 383, ...,   0,   0,   0],
       [  4, 675,   3, ...,   0,   0,   0],
       [  9,  27,   8, ...,   0,   0,   0],
       ...,
       [ 98, 251,   8, ...,   0,   0,   0],
       [ 19,  12,   3, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int32)

In [ ]:
words = list(word2index.values())

In [ ]:
enc_que = []
enc_ans = []
# runs for 6 mins
for sen in encoder_input_data:
  senvec = [embedding_matrix[w] for w in sen if w in words]
  enc_que.append(senvec)
  #break

In [ ]:
#runs for 6 min find a way to save
for sen in decoder_input_data:
  senvec = [embedding_matrix[w] for w in sen if w in words]
  enc_ans.append(senvec)

In [ ]:
questions = np.array(encoder_input_data,dtype=np.float64)
answers = np.array(decoder_input_data,dtype=np.float64)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(questions, answers, train_size=0.8) 

In [ ]:
X_train.shape

(8000, 20)

In [ ]:
len(embedding_matrix)

2005

In [ ]:
from tensorflow.keras.layers import Embedding, SimpleRNN
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.models import Model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics='accuracy')

In [ ]:
X_train.shape

(8000, 20)

In [ ]:
X = X_train.reshape(len(X_train),20,1)

In [ ]:
y_train.shape

(8000, 20)

In [ ]:
model.fit(X, # for stateful we need 
          y_train, # number of samples divisible by the batch size
          epochs=500, 
          batch_size=batch_size,
          validation_split = 0.2, 
          shuffle= True # don't shuffle if stateful 
         )

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train_pad = [to_categorical(j, num_classes=2005) for j in y_train]

In [ ]:
input = Input(shape=(20,))
model = Embedding(input_dim=2005,output_dim=200,input_length=20)(input)
model = SimpleRNN(64, return_sequences = True)(model)
model = SimpleRNN(64, return_sequences = True)(model)

out = Dense(2005,activation='softmax')(model)
model = Model(input,out)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 20, 200)           401000    
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 20, 64)            16960     
_________________________________________________________________
simple_rnn_10 (SimpleRNN)    (None, 20, 64)            8256      
_________________________________________________________________
dense_3 (Dense)              (None, 20, 2005)          130325    
Total params: 556,541
Trainable params: 556,541
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,np.array(y_train_pad),validation_split=0.2, epochs=10)

Epoch 1/10
200/200 [==============================] - 13s 56ms/step - loss: 3.8505 - accuracy: 0.5064 - val_loss: 3.0368 - val_accuracy: 0.5422
Epoch 2/10
200/200 [==============================] - 11s 54ms/step - loss: 2.9701 - accuracy: 0.5467 - val_loss: 2.9611 - val_accuracy: 0.5481
Epoch 3/10
200/200 [==============================] - 11s 54ms/step - loss: 2.9217 - accuracy: 0.5518 - val_loss: 2.9659 - val_accuracy: 0.5501
Epoch 4/10
200/200 [==============================] - 11s 54ms/step - loss: 2.8981 - accuracy: 0.5534 - val_loss: 2.9597 - val_accuracy: 0.5502
Epoch 5/10
200/200 [==============================] - 11s 54ms/step - loss: 2.8784 - accuracy: 0.5533 - val_loss: 2.9661 - val_accuracy: 0.5507
Epoch 6/10
200/200 [==============================] - 11s 55ms/step - loss: 2.8544 - accuracy: 0.5536 - val_loss: 2.9888 - val_accuracy: 0.5501
Epoch 7/10
200/200 [==============================] - 11s 55ms/step - loss: 2.8403 - accuracy: 0.5537 - val_loss: 3.0019 - val_accuracy: